# Model Fitting with TensorFlow/Keras

This notebook:
1. Loads prepared data from parquet
2. Trains a neural network model using TensorFlow/Keras
3. Exports the model to ONNX format
4. Saves the model as TensorFlow SavedModel

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import tf2onnx
import onnx
import os


In [ ]:
print(f"TensorFlow version: {tf.__version__}")

## Load Data from Parquet

In [ ]:
df = pd.read_parquet("../data/processed/prepared_data.parquet")

In [ ]:
print(f"Data shape: {df.shape}")

In [ ]:
df.head()

In [ ]:
df.info()

## Prepare Features and Labels

In [ ]:
# Separate features (X) and target (y)
X = df[['A', 'B']].values.astype(np.float32)
y = df['Output'].values.astype(np.float32)

print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"\nFeatures:\n{X}")
print(f"\nLabels: {y}")

## Build the Neural Network Model

In [ ]:
# Build a simple neural network for XOR problem
model = keras.Sequential([
    keras.layers.Dense(2, activation='tanh', input_shape=(2,), name='hidden_1'),
    keras.layers.Dense(2, activation='tanh', name='hidden_2'),
    keras.layers.Dense(1, activation='sigmoid', name='output')

], name='xor_model')

model.compile(
    optimizer=Adam(learning_rate=0.1),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

## Train the Model

In [ ]:
history = model.fit(
    X, y,
    epochs=150,
    batch_size=4,
    verbose=1
)

print(f"Final Loss: {history.history['loss'][-1]:.6f}")
print(f"Final Accuracy: {history.history['accuracy'][-1]:.6f}")

## Evaluate Model Predictions

In [ ]:
# Make predictions and compare with actual values
predictions = model.predict(X, verbose=0)
predicted_classes = (predictions > 0.5).astype(int).flatten()

print("Input A | Input B | Expected | Predicted | Raw Output")
print("-" * 55)
for i in range(len(X)):
    print(f"   {int(X[i,0])}    |    {int(X[i,1])}    |    {int(y[i])}     |     {predicted_classes[i]}     |   {predictions[i,0]:.4f}")

## Save Model as TensorFlow SavedModel

### Prepare Models directory

In [ ]:
# Create models directory if it doesn't exist
models_dir = "../models"
os.makedirs(models_dir, exist_ok=True)

### Save in Native Keras Format

In [ ]:
# Also save in native Keras format
keras_path = os.path.join(models_dir, "xor_model.keras")
model.save(keras_path)
print(f"Keras model saved to: {keras_path}")

### Save as TensorFlow SavedModel

In [ ]:
savedmodel_path = os.path.join(models_dir, "xor_model_savedmodel")
model.export(savedmodel_path, format='tf_saved_model')
print(f"TensorFlow SavedModel saved to: {savedmodel_path}")

### Export Model to ONNX Format

In [ ]:
# Convert to ONNX format using Keras export
onnx_path = os.path.join(models_dir, "xor_model.onnx")

# Export to ONNX using Keras 3's built-in export
model.export(onnx_path, format='onnx')
print(f"ONNX model saved to: {onnx_path}")

### Export Model to TFLite (Mobile) Format

In [ ]:
# Convert to TFLite format
tflite_path = os.path.join(models_dir, "xor_model.tflite")

# Create TFLite converter from Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Convert the model
tflite_model = converter.convert()

# Save TFLite model
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved to: {tflite_path}")
print(f"TFLite model size: {len(tflite_model):,} bytes")

### TensorRT (GPU serving)
TensorRT Requirements:

- Requires NVIDIA GPU with CUDA support
- Needs tensorrt package installed (typically via NVIDIA's repositories)
`pip install tensorrt`

## Verify Saved Models

In [ ]:
# List saved models
print("Saved model files:")
for root, dirs, files in os.walk(models_dir):
    level = root.replace(models_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        filepath = os.path.join(root, file)
        size = os.path.getsize(filepath)
        print(f"{subindent}{file} ({size:,} bytes)")

## Verify ONNX Model

In [ ]:
# Verify ONNX model
onnx_model_loaded = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model_loaded)
print("✓ ONNX model is valid!")

# Print ONNX model info
print(f"\nONNX Model Info:")
print(f"  IR Version: {onnx_model_loaded.ir_version}")
print(f"  Opset Version: {onnx_model_loaded.opset_import[0].version}")
print(f"  Producer: {onnx_model_loaded.producer_name}")

print(f"\nInputs:")
for inp in onnx_model_loaded.graph.input:
    dims = [dim.dim_value if dim.dim_value else 'batch' for dim in inp.type.tensor_type.shape.dim]
    print(f"  - {inp.name}: {dims}")

print(f"\nOutputs:")
for output in onnx_model_loaded.graph.output:
    dims = [dim.dim_value if dim.dim_value else 'batch' for dim in output.type.tensor_type.shape.dim]
    print(f"  - {output.name}: {dims}")


## Model Export Summary

In [ ]:
print(f"✓ TensorFlow SavedModel: {savedmodel_path}")
print(f"✓ Keras Model (.keras): {keras_path}")
print(f"✓ ONNX Model: {onnx_path}")
print(f"✓ TFLite Model (mobile): {tflite_path}")